In [1]:
import numpy as np
import pandas as pd
import jieba
import time
import re
from collections import Counter

In [2]:
obj_feat_data = pd.read_csv('./obj_feat_data.csv', encoding='utf-8')
obj_feat_data.head()

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (15,75,76) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,vid,0102,2302,0113,0114,0116,1001,3192,3191,3197,...,3432,1105,269044,1104,3868,3855,669032,A502,21A014,2285
0,000330ad1f424114719b7525f400660b,甲状腺彩超（含颈部淋巴细胞）未发现明显异常$前列腺:前列腺稍大膀胱、双侧输尿管未发现明显异常...,健康,肝脏大小、形态正常，包膜光整，肝内血管走行较清晰，回声均匀。,胆囊大小、形态正常，囊壁光整，囊腔内透声好，胆总管无扩张。,脾脏大小、形态正常，包膜光整，回声均匀。,窦性心律肢体导联低电压,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000381f0069cbf7537e6aac8923034ae,左肾、右肾、膀胱、双侧输尿管未发现明显异常$子宫:绝经后子宫左附件、右附件未发现明显异常$肝...,健康,肝脏形态大小正常，轮廓规整，肝内回声呈点状密集弥漫性增强，肝内管道结构尚清晰。,胆囊大小、形态正常，囊壁光整，囊腔内透声好，胆总管无扩张。,脾脏大小、形态正常，包膜光整，回声均匀。,窦性心律正常心电图,阴性,阴性,阴性,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0003848ebd8d8163603760d53d975693,"左肾、右肾、膀胱、双侧输尿管未发现明显异常$双侧乳腺小叶增生$甲状腺双叶回声不均匀,建议甲功...",健康,肝脏大小、形态正常，包膜光整，肝内血管走行较清晰，光点分布尚均匀，其内未见明显异常光团。,胆囊大小、形态正常，囊壁光整，囊腔内透声好，胆总管无扩张。,脾脏大小、形态正常，包膜光整，内光点均匀。,窦性心律心电轴右偏,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000401cbf304d5a8bd862a81bacfa494,肝:脂肪肝（轻度）胆、胰、脾、左肾、右肾未发现明显异常$双侧乳腺轻度增生$子宫:宫颈囊肿（多...,健康,肝脏形态大小正常，轮廓规整，肝内回声呈点状密集弥漫性增强，肝内管道结构尚清晰。,胆囊大小正常，壁光滑，腔内暗区清晰，胆总管测值正常范围。,脾脏大小测值正常，回声均匀，脾静脉测值正常。,NaN,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00044a586c249c05f0969e45ef03ab9d,"双侧乳腺小叶增生右乳腺低回声区，考虑结节,建议医院进一步检查$肝、胆、胰、脾未发现明显异常$...",健康,肝脏大小、形态正常，包膜光整，肝内血管走行较清晰，光点分布尚均匀，其内未见明显异常光团。,胆囊大小、形态正常，囊壁光整，囊腔内透声好，胆总管无扩张。,脾脏大小、形态正常，包膜光整，内光点均匀。,窦性心律，正常心电图,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
obj_feat_data.shape

(57298, 226)

In [4]:
def get_stop_words(file_path):
    with open('stopwords_t.csv', 'r', encoding='utf-8') as f:
        lines = f.readlines()
        stop_words = set()
        for line in lines:
            line = line.strip()
            stop_words.add(line)
        stop_words = list(stop_words)
    return stop_words

In [5]:
punctuation = '[’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~（）；，、。？【】—·0-9 ]+'             #'[]’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~（）、。！？：；‘’，｛【】《》｝'
stop_words = get_stop_words('stopwords_t.csv')
for i in punctuation:
    stop_words.append(i)
stop_words[0:10]

['', '大量', '彻底', '假若', '风雨无阻', '一天', '不然的话', '理当', '并肩', '赶快']

In [6]:
def get_obj_feat_df(feat):
    # 用来存储所有的词以及出现的次数
    all_words = Counter()
    # 用来存储每条切分后的数据
    all_split_words = []
    for line in obj_feat_data[feat]:
        try:
            line = re.sub(punctuation, '', line) 
            line = jieba.cut(line, cut_all=False)
            line = list(line)
            all_split_words.append(line)
            for w in line:
                all_words[w] += 1        
        except:
            all_split_words.append(np.nan)
            
    # 选取出现频次高于5次的词语
    # 词袋
    word_bag = set()
    for word, count in all_words.items():
        if count > 5:
            word_bag.add(word)
     
    # 构建每条数据的对应词频字典
    # 用来存储每条数据的（词语：词频）字典
    feat_data = []
    for i in all_split_words:
        dict_i = {}
        try:
            for w in i:
                if w in word_bag:
                    if w in dict_i:
                        dict_i[w] += 1
                    else:
                        dict_i[w] = 1
        except:
            pass
        feat_data.append(dict_i)

    feat_data_df = pd.DataFrame(feat_data)
    
    return feat_data_df

In [ ]:
obj_feats = [x for x in obj_feat_data.columns if x not in ['vid']]
len(obj_feats)

225

In [ ]:
obj_feat_df_new = pd.DataFrame()
count = 0
start = time.time()
for feat in obj_feats:
    count += 1
    feat_df = get_obj_feat_df(feat)
    obj_feat_df_new = pd.concat([obj_feat_df_new, feat_df], axis = 1)
    print(count)
    print(obj_feat_df_new.shape)
    print('*******************************')
    
end = time.time()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\CHENBI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.975 seconds.
Prefix dict has been built succesfully.


1
(57298, 536)
*******************************
2
(57298, 540)
*******************************
3
(57298, 1042)
*******************************
4
(57298, 1363)
*******************************
5
(57298, 1477)
*******************************
6
(57298, 1753)
*******************************
7
(57298, 1758)
*******************************
8
(57298, 1763)
*******************************
9
(57298, 1766)
*******************************
10
(57298, 1770)
*******************************
11
(57298, 1775)
*******************************
12
(57298, 2088)
*******************************
13
(57298, 2411)
*******************************
14
(57298, 2417)
*******************************
15
(57298, 2417)
*******************************
16
(57298, 2422)
*******************************
17
(57298, 2513)
*******************************
18
(57298, 2526)
*******************************
19
(57298, 2535)
*******************************
20
(57298, 4376)
*******************************
21
(57298, 4713)
**************